# Brace Analysis Version 2
The photo shows the enf\d details of a typical brace in a 4-storey steel structure.  This was photographed in Ottawa, in September, 2015.

This notebook shows all of the computations necessary to compute $T_r$ for a similar brace.  **Caution**, all of the dimensions and properties were invented by the author of this notebook; no attempt has been made to have an accurate model of the real structure.

![Brace End Details](brace.jpg)

![Overall Sketch](brace-sketch.svg)

In [1]:
from __future__ import print_function, division

In [2]:
from Designer import Data,Designer
import math
sind = lambda t: math.sin(math.radians(t))
cosd = lambda t: math.cos(math.radians(t))

ImportError: cannot import name 'Data'

In [ ]:
def Warn(s):
    print('***** WARNING:',s,'*****')

class Data(object):
    
    """DATA root class."""
    
    __CAUTIOUS__ = True
    
    @classmethod
    def get(self,fields):
        fs = [x.strip() for x in fields.split(',') if x.strip()]
        if self.__CAUTIOUS__:
            dicts = [(x,vars(x)) for x in self.mro() if x not in [Data,object]]
            for f in fs:
                clses = [x for x,v in dicts if f in v]
                if len(clses) > 1:
                    Warn("Field '{}' is defined in multiple classes: {}".
                          format(f,', '.join([c.__name__ for c in clses])))
        vs = [getattr(self,f) for f in fs]
        if len(fs) == 1:
            return vs[0]
        return vs

In [ ]:
import sst
SST = sst.SST()

In [ ]:
phiw = 0.67
phiu = 0.75
phib = 0.80
phibr = 0.80
phi = 0.90

In [ ]:
des = Designer(title='Typical Cross Brace',var='Tr',units='kN',trace=True)

# useful abbreviations:
REQ = des.require       # a requirement
CHK = des.check         # a check
REC = des.record        # record calculation details

In [ ]:
class Bolts(Data):
    grade = 'ASTM A325M'
    size = 'M20'
    Fu = 830.
    d = 20.
    holes = 'punched'
    hd = d + 2 + (0 if holes == 'drilled' else 2)
    threads_intercepted = True
    nlines = 2
    nacross = 3
    g = 75.
    s = 75.

class Welds(Data):
    grade = 'E49xx'
    Xu = 490.
    matching = True

class Plate_Steel(Data):
    grade = 'CSA G40.21 350W'
    Fy = 350.
    Fu = 450.
    
class HSS_Steel(Data):
    grade = 'CSA G40.21 350W'
    Fy = 350.
    Fu = 450.
    
class HSS(HSS_Steel):
    size = 'HS127x127x13'
    
class Tongue(Plate_Steel,Bolts):
    T = 20.
    W = 280.
    L = 260.
    e = 40.

## Gusset Plate Details
![Gusset Plate Details](gusset-details.svg)

In [ ]:
class Gusset(Plate_Steel,Bolts,Welds):
    W1 = Tongue.W
    W2 = 110.
    T = Tongue.T
    e = 40.
    D = 8.
    theta = 45.

### Gusset to HSS Weld

In [ ]:
w1,w2,D,theta,Xu = Gusset.get('W1,W2,D,theta,Xu')

L1 = w2+w1*cosd(theta)
L = (L1/sind(theta))*cosd(theta) + w1*sind(theta)

Mw = 1.0
Aw = 2*L*.707*D
Vr = 0.67*phiw*Aw*Xu*(1+sind(theta)**1.5)*Mw * 1E-3

REC('Gusset to HSS Weld','L,Xu,w1,w2,D,theta,Aw,Mw,phiw',Tr=Vr)

### Gusset Block Shear
#### Case 1)

In [ ]:
nlines,nacross,g,s,e,t,hd,Fy,Fu = Gusset.get('nlines,nacross,g,s,e,T,hd,Fy,Fu')

An = t*((nacross-1)*g - (nacross-1)*hd)
Agv = 2*t*((nlines-1)*s + e)
Ut = 1.0
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Gusset Block Shear 1)','An,Agv,Ut',Tr=Vr)

#### Case 2)

In [ ]:
An = t*((nacross-1)*g + ((w1/2.)-(nacross-1)*g/2.))
Agv = t*((nlines-1)*s + e)
Ut = 0.8
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Gusset Block Shear 2)','An,Agv,Ut',Tr=Vr)

#### Tearout

In [ ]:
Agv = t*((nlines-1)*s + e) * nacross * 2
Vr = phiu * 0.6*Agv*(Fy+Fu)/2. * 1E-3
REC('Gusset tearout','Agv',Tr=Vr)

In [ ]:
#################################################################### converted to here

## Lap Plate
### Lap Plate Details
Consider the thickness of both plates together when computing the resistance.
As the plate is symmetric, we only have to investigate one end.
![Lap Plate Details](lap-plate-details.svg)

In [ ]:
LapPlate = Data(W = 230.,
                L = 315.,
                t = 10.+10.,     # include 2 plates
                e = 40.,         # could be different than gusset
                g = Gusset.g,    # must be the same as gusset plate
                s = Gusset.s,
                nlines = Gusset.nlines,
                nacross = Gusset.nacross,
               )

### Lap Plate: Gross Section Yielding

In [ ]:
wg,t = LapPlate['W,t']
Fy = PL.Fy
Ag = wg*t
Tr = phi*Ag*Fy * 1E-3
REC('Lap Plate, Gross Yield','Ag,Fy,phi',Tr=Tr)

### Lap Plate: Net Section Fracture

In [ ]:
wg,t,nacross = LapPlate['W,t,nacross']
hd = Bolt.hd
Fu = PL.Fu

wn = wg - nacross*hd
Ane = An = wn*t
Tr = phiu*Ane*Fu * 1E-3
REC('Lap Plate, Net Fracture','Fu,phiu,Ane',Tr=Tr)

### Lap Plate: Block Shear
#### Case 1)

In [ ]:
g,t,nacross,nlines,e,s = LapPlate['g,t,nacross,nlines,e,s']
hd = Bolt.hd
Fy,Fu = PL['Fy,Fu']

An = ((nacross-1)*g - (nacross-1)*hd)*t
Agv = (e + (nlines-1)*s)*t*2
Ut = 1.0
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Lap Plate Block Shear 1)','An,Agv,Ut',Tr=Vr)

#### Case 2)

In [ ]:
g,t,nacross,nlines,e,s,wg = LapPlate['g,t,nacross,nlines,e,s,W']
hd = Bolt.hd
Fy,Fu = PL['Fy,Fu']

edge = (wg - (nacross-1)*g)/2.0
An = (wg - edge - (nacross-0.5)*hd)*t
Agv = (e + (nlines-1)*s)*t
Ut = 0.8
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Lap Plate Block Shear 2)','An,Agv,Ut',Tr=Vr)

#### Tearout

In [ ]:
t,nlines,nacross,e,s = LapPlate['t,nlines,nacross,e,s']
Fy,Fu = PL['Fy,Fu']

An = 0
Agv = (e + (nlines-1)*s)*t*2*nacross
Ut = 1
Vr = phiu*(Ut*An*Fu + 0.6*Agv*(Fy+Fu)/2.) * 1E-3
REC('Lap Plate tearout','Agv',Tr=Vr)

## Tongue Plate
### Tongue Plate Details
![Tongue Plate Details](tongue-plate-details.svg)

In [ ]:
Tongue = Data(W = Gusset.w1,
              t = Gusset.t,
              D = 8.,
              nlines = Gusset.nlines,
              nacross = Gusset.nacross,
              e = 40.,
              s = Gusset.s,
              g = Gusset.g,
              c = 45.,
              L = 260.              
             )
Tongue.Lw = Tongue.L - (Tongue.c + (Tongue.nlines-1)*Tongue.s + Tongue.e)
Tongue.Dh = SST.section(HSS.size,'D')

In [ ]:
vars(Tongue)

### Tongue Plate - Bolted End
#### Gross Section Yield

In [ ]:
wg,t = Tongue['W,t']
Fy = PL['Fy']
Ag = wg*t
Tr = phi*Ag*Fy * 1E-3
REC('Tongue Plate, Gross Yield','Ag',Tr=Tr)

#### Net Section Fracture

In [ ]:
wg,t,n = Tongue['W,t,nacross']
Fu = PL['Fu']
hd = Bolt['hd']

wn = wg - n*hd
Ane = An = wn*t
Tr = phiu*An*Fu * 1E-3
REC('Tongue Plate, Net Area Fracture (Bolted)','An',Tr=Tr)

#### Block Shear
There is no need to check block shear.  Of necessity, the thickness of the tongue plate is the same as
the gusset plate, and as the widths and bolting details are also the same, the capacities with respect to block shear
will also be the same.

### Tongue Plate - Welded End
#### Shear Lag - Effective Net Area

In [ ]:
w,w2,L,t = Tongue['W,Dh,Lw,t']
Fu = PL.Fu
if L >= 2*w2:
    An2 = 1.00*w2*t
elif L >= w2:
    An2 = 0.5*w2*t + 0.25*L*t
else:
    An2 = 0.75*L*t
    
n,g = Tongue['nacross,g']
w3 = (w-w2)/2.
xbar = w3/2.
if L >= w3:
    An3 = (1.-xbar/L)*w3*t
else:
    An3 = 0.50*L*t
Ane = An2 + An3 + An3
Tr = phiu*Ane*Fu * 1E-3
REC('Tongue Plate, Net Area Fracture (Welded)','w2,w3,An2,An3,Ane',Tr=Tr)

## Fasteners
### Bolts
#### Shear

In [ ]:
d,Fu,threads_intercepted = Bolt['d,Fu,threads_intercepted']
t = min(Gusset.t,Tongue.t,2*LapPlate.t)
n = min(Gusset.nlines*Gusset.nacross,Tongue.nlines*Tongue.nacross)
m = 2
Ab = 3.14159*d*d/4.
Vr = 0.6*phib*n*m*Ab*Fu * 1E-3
if threads_intercepted:
    Vr *= 0.7
REC('Bolts in Shear','n,m,d,Ab,Fu,t',Tr=Vr)

#### Bearing

In [ ]:
Br = 3*phibr*n*t*d*Fu * 1E-3
REC('Bolts in Bearing','n,t,d,Fu',Tr=Br)

### Welds - HSS to Tongue Plate

In [ ]:
L,D = Tongue['Lw,D']
Xu,matching = Weld['Xu,matching']
Aw = 4.*L*D*0.707
Vr = 0.67*phiw*Aw*Xu * 1E-3
if matching:
    REC('Weld metal (HSS to Plate)','D,L,Aw,Xu',Tr=Vr)
else:
    Fu = min(HSS.Fu,PL.Fu)
    Am = 4.*L*D
    Vr = min(Vr,0.67*phiw*Am*Fu)
    REC('Weld/Base metal (HSS to Plate)','D,L,Aw,Am,Xu,Fu',Tr=Vr)

## HSS + Cover Plate
![HSS Details](hss-details.svg)
Each half of the HSS is a plate of width w (the meridian distance around) welded on 2 edges (perpendicular
to the figure).  The cover plate is an additional plate welded on two edges.
### Net Section Fracture

In [ ]:
D,t,Ag = SST.section(HSS.size,'D,T,A')
hFy,hFu = HSS['Fy,Fu']
tt,L = Tongue['t,Lw']
w = (4.*(D-t) - 2*tt)/2.   # meridian distance around 1/2 of cut HSS
if L >= 2*w:
    An2 = 1.00*w*t
elif L >= w:
    An2 = 0.5*w*t + 0.25*L*t
else:
    An2 = 0.75*L*t
hAne = An2
hTr = 2*phiu*hAne*hFu * 1E-3

cw,ct,cFy,cFu = Cover['w,t,Fy,Fu']  # cover plate
if L >= 2*cw:
    cAn2 = 1.00*cw*ct
elif L >= cw:
    cAn2 = 0.5*cw*ct + 0.25*L*ct
else:
    cAn2 = 0.75*L*ct
cAne = cAn2
cTr = 2*phiu*cAne*cFu * 1E-3
Tr = hTr + cTr
REC('HSS, net section fracture','w,t,hAne,hTr,cw,cAne,cTr',Tr=Tr)

### Gross Section Yield

In [ ]:
Tr = phi*Ag*Fy  * 1E-3
REC('HSS Gross Section Yield','Ag',Tr=Tr)

## Cover Plate to HSS Weld
Ensure that the length of the weld can develop the full strength of the cover plate.

In [ ]:
cTr = phiu*cAne*cFu * 1E-3
L = Tongue.Lw
D = Cover.D
Xu = Weld.Xu
Aw = 2*L*D*0.707
wTr = 0.67*phiw*Aw*Xu * 1E-3
CHK(wTr>=cTr,'Coverplate weld strength'.format(wTr,cTr),'wTr,cTr,L,D,Aw')

## To Do
* Check that gusset plate doesn't cause undo flexural problems in the flange of the HSS column member.

## Summary

In [ ]:
des.summary()